In [38]:
import os
import time
import torch
from torch import nn, optim
from torch import tensor
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from skimage.filters import gabor
from skimage.feature import hog
#import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import r2_score, precision_score, recall_score, f1_score, confusion_matrix

In [4]:
class FaceDataset(Dataset):
    def __init__(self, index_file):
        self.df = pd.read_csv(open(index_file))
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i):
        Y = tensor(self.df.iloc[i][0])
        X = tensor(self.df.iloc[i][1:])
        return (X, Y)

In [5]:
train_data = FaceDataset('play.csv')
batch_size = 32
train_data_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=4)

In [42]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.fc1 = nn.Linear(2304, 100)
        self.fc2 = nn.Linear(100, 7)
        self.output = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(-1, 2304)
        tmp = []
        for i in range(x.shape[0]):
            image = x[i]
            gabor_real, gabor_imag = gabor(image.reshape(48, 48), frequency=1.0)
            gabor_real = tensor(gabor_real.ravel())
            gabor_imag = tensor(gabor_imag.ravel())
            hog_ = tensor(hog(image.reshape(48, 48)))
            tmp.append(hog_)
            #tmp.append(torch.cat((image, tensor(gabor_real.ravel()), tensor(gabor_imag.ravel()))))
        f0 = torch.stack(tmp)
        f1 = F.relu(self.fc1(x))
        f2 = F.relu(self.fc2(f1))
        return self.output(f2)

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#torch.cuda.set_device(0)

net = NNet()
net = net.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)
print(net)

cpu
NNet(
  (fc1): Linear(in_features=2304, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=7, bias=True)
  (output): Softmax(dim=1)
)


In [44]:
total_loss = 0
for epoch in range(10):
    for i, data in enumerate(train_data_loader, 0):
        images, classes = data
        images, classes = images.to(device).float(), classes.to(device).long()
        #images, classes = images.float(), classes.long()
        optimizer.zero_grad()
        output = net(images)
        #print(output)
        
        loss = loss_function(output, classes)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() 
        if(i % 20 == 19):
            print("After epoch {}, mini-batch {}, loss: {}".format(epoch + 1, i + 1, total_loss))
            correct = 0
            total = 0
            total_loss = 0
    total_loss = 0

After epoch 1, mini-batch 20, loss: 38.77716827392578
After epoch 2, mini-batch 20, loss: 37.94069194793701
After epoch 3, mini-batch 20, loss: 37.35272550582886
After epoch 4, mini-batch 20, loss: 36.86849617958069
After epoch 5, mini-batch 20, loss: 36.28807735443115
After epoch 6, mini-batch 20, loss: 35.91375195980072
After epoch 7, mini-batch 20, loss: 35.78340291976929
After epoch 8, mini-batch 20, loss: 35.302658796310425
After epoch 9, mini-batch 20, loss: 35.15214133262634
After epoch 10, mini-batch 20, loss: 34.691609144210815


In [45]:
train_data = FaceDataset('public_test.csv')
batch_size = 32
test_data_loader = DataLoader(train_data, batch_size, shuffle=False, num_workers=2)

In [46]:
actual, prediction = [], []
for i, data in enumerate(test_data_loader, 0):
    images, classes = data
    #images, classes = images.to(device).float(), classes.to(device).long()
    images, classes = images.float(), classes.long()
    actual.extend(classes.squeeze().tolist())
    
    optimizer.zero_grad()
    output = net(images)
    
    prediction.extend(torch.argmax(output, dim=1).squeeze().tolist())
    #print(output)
print(f1_score(actual, prediction, average='macro'))
print(confusion_matrix(actual, prediction))

0.22281345674057343
[[ 80   0  98 290 167 131 126]
 [  7   0  16  32  10  18  15]
 [ 38   0 117 251 148 218 147]
 [115   0 105 871 144 175 211]
 [ 82   0 138 312 241 122 202]
 [ 26   0  95 139  55 296 106]
 [ 31   0 105 370 160 142 306]]


In [47]:
print(np.bincount(np.array(actual)))
print(np.bincount(np.array(prediction)))

[ 892   98  919 1621 1097  717 1114]
[ 379    0  674 2265  925 1102 1113]
